In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import tensorflow as tf   
import tflearn            
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sikha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sikha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# importing our intent file used for training the model.
with open("intents.json") as json_data: 
    intents = json.load(json_data) 

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Hey',
    'Ekse',
    'Hi'],
   'responses': ['Hi stranger',
    'how can I help',
    'Ey watsupp',
    'namaste',
    'Hola',
    'Hey dude',
    'You again'],
   'context': ['']},
  {'tag': 'casual',
   'patterns': ['How are you',
    'are you okay?',
    'how are you doing?',
    'is everything alright'],
   'response': ['i am doing fine and hope you are too! please raise your concerns now so that i can help you with that'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'Ok bye', 'Bye Bye'],
   'responses': ['See you!',
    'Have a nice day',
    'Sure Bye',
    'Later dude',
    'Sayoonara',
    'Peace out',
    'Sure dude',
    'Ciao'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
 

In [4]:
# Empty lists for appending the data after processing NLP
words=[]
documents = []
classes = []


# This list will be used for ignoring all unwanted punctuation marks.
ignore = ["?"]

# Starting a loop through each intent in intents["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # tokenizing each and every word in the sentence by using word tokenizer and storing in w
        w = nltk.word_tokenize(pattern) 
        #print(w)
        
        # Adding tokenized words to words empty list that we created
        words.extend(w) 
        #print(words)
        
        # Adding words to documents with tag given in intents file
        documents.append((w, intent["tag"]))
        #print(documents)
        
        # Adding only tag to our classes list
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  #If tag is not present in classes[] then it will append into it.
            #print(classes)

In [5]:
#Performing Stemming by using stemmer.stem() nd lower each word 
#Running loop in words[] and ignoring punctuation marks present in ignore[]

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))  #Removing Duplicates in words[]

#Removing Duplicate Classes
classes = sorted(list(set(classes)))

#Printing length of lists we formed
print(len(documents),"Documents \n")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

84 Documents 

26 classes ['HR_related_problem', 'Location', 'casual', 'connect_people', 'customer_satisfaction', 'email_id', 'forgot_password', 'goodbye', 'greeting', 'highest_grossing', 'invalid', 'key_customers', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'opentoday', 'options', 'order_components', 'order_tracking', 'payments', 'solve_problems', 'supplier_info', 'thanks', 'version_update']
164 unique stemmed words ["'s", ',', '.', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'about', 'abx', 'accept', 'accident', 'alright', 'am', 'analysis', 'and', 'anyone', 'are', 'awesome', 'based', 'be', 'benefit', 'bored', 'bye', 'can', 'card', 'cash', 'chat', 'clarity', 'clear', 'company', 'compensation', 'component', 'consists', 'could', 'credit', 'customer', 'date', 'day', 'delivery', 'demand', 'department', 'design', 'development', 'do', 'doing', 'ekse', 'employee', 'everything', 'feedback', 'find', 'for', 'forgets', 'forgot', 'gmail

In [6]:
#Creating Training Data which will be furthur used for training
training = []
output = []

#Creating empty array for output
output_empty = [0] * len(classes)

#Creating Training set and bag of words for each sentence
for doc in documents:
    bag = [] #Initialising empty bag of words

    pattern_words = doc[0] #Storing list of tokenized words for the documents[] tp pattern_words
    #print(pattern_words)
    
    #Again Stemming each word from pattern_words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]  
    #print(pattern_words)
    
    #Creating bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #It will give output 1 for curent tag and 0 for all other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [8]:
random.shuffle(training) #Suffling the data or features
training = np.array(training,dtype=object) #Converting training data into numpy array

#Creating Training Lists
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Train on 84 samples
Epoch 1/200
84/84 [==============================] - 3s 32ms/sample - loss: 3.2721 - acc: 0.0714
Epoch 2/200
84/84 [==============================] - 0s 666us/sample - loss: 3.2573 - acc: 0.0714
Epoch 3/200
84/84 [==============================] - 0s 571us/sample - loss: 3.1368 - acc: 0.1548
Epoch 4/200
84/84 [==============================] - 0s 571us/sample - loss: 3.0894 - acc: 0.1190
Epoch 5/200
84/84 [==============================] - 0s 761us/sample - loss: 3.0167 - acc: 0.2143
Epoch 6/200
84/84 [==============================] - 0s 857us/sample - loss: 2.8937 - acc: 0.2143
Epoch 7/200
84/84 [==============================] - 0s 856us/sample - loss: 2.8429 - acc: 0.1905
Epoch 8/200
84/84 [==============================] - 0s 761us/sample - loss: 2.7478 - acc: 0.2381
Epoch 9/200
84/84 [==============================] - 0s 761us/sample - loss: 2.5527 - acc: 0.3333
Epoch 10/200
84/84 [==============================] - 0s 857us/sample - loss: 2.5318 - acc: 0.2857


84/84 [==============================] - 0s 571us/sample - loss: 0.2134 - acc: 0.9286
Epoch 84/200
84/84 [==============================] - 0s 476us/sample - loss: 0.1825 - acc: 0.9524
Epoch 85/200
84/84 [==============================] - 0s 571us/sample - loss: 0.2114 - acc: 0.9524
Epoch 86/200
84/84 [==============================] - 0s 571us/sample - loss: 0.1035 - acc: 0.9762
Epoch 87/200
84/84 [==============================] - 0s 571us/sample - loss: 0.2133 - acc: 0.9524
Epoch 88/200
84/84 [==============================] - 0s 666us/sample - loss: 0.1218 - acc: 0.9643
Epoch 89/200
84/84 [==============================] - 0s 571us/sample - loss: 0.1701 - acc: 0.9524
Epoch 90/200
84/84 [==============================] - 0s 571us/sample - loss: 0.1617 - acc: 0.9524
Epoch 91/200
84/84 [==============================] - 0s 571us/sample - loss: 0.1526 - acc: 0.9524
Epoch 92/200
84/84 [==============================] - 0s 571us/sample - loss: 0.1953 - acc: 0.9405
Epoch 93/200
84/84 [===

84/84 [==============================] - 0s 571us/sample - loss: 0.0628 - acc: 0.9762
Epoch 166/200
84/84 [==============================] - 0s 666us/sample - loss: 0.0791 - acc: 1.0000
Epoch 167/200
84/84 [==============================] - 0s 666us/sample - loss: 0.1561 - acc: 0.9643
Epoch 168/200
84/84 [==============================] - 0s 666us/sample - loss: 0.1008 - acc: 0.9762
Epoch 169/200
84/84 [==============================] - 0s 666us/sample - loss: 0.0910 - acc: 0.9881
Epoch 170/200
84/84 [==============================] - 0s 571us/sample - loss: 0.0969 - acc: 0.9524
Epoch 171/200
84/84 [==============================] - 0s 571us/sample - loss: 0.0984 - acc: 0.9643
Epoch 172/200
84/84 [==============================] - 0s 666us/sample - loss: 0.1401 - acc: 0.9405
Epoch 173/200
84/84 [==============================] - 0s 666us/sample - loss: 0.0494 - acc: 1.0000
Epoch 174/200
84/84 [==============================] - 0s 666us/sample - loss: 0.1339 - acc: 0.9524
Epoch 175/200


In [10]:
#Importing pickle module
import pickle

#Dumping training data by using dump() and writing it into training_data in binary mode
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [11]:
#Restoring all data structure

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [12]:
with open("intents.json") as json_data:
    intents = json.load(json_data)  #Loading our json_data

In [13]:
# Loading the saved model
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5') #Loading training model which we saved


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
#Cleaning User Input
def clean_up_sentence(sentence):
    
    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence
    
    #Stemming each word from the user's input
    sentence_words= [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

#Returning bag of words array: 0 or 1 or each word in the bag that exists in as we have declared in above lines
def bow(sentence, words, show_details=False):
    
    #Tokenizing the user input
    sentence_words = clean_up_sentence(sentence)
    
    #Generating bag of words from the sentence that user entered
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [15]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
        else:
            result = "You must ask the right questions"
    return result


    
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res


In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("Hi there! how can i help you?")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()